Welcome to exercise one of week four of “Apache Spark for Scalable Machine Learning on BigData”. In this exercise we’ll work on classification.

Let’s create our DataFrame again:

In [1]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200219191651-0000
KERNEL_ID = e0951f42-b86e-4aa8-b002-952f851f6c78
--2020-02-19 19:16:54--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet [following]
--2020-02-19 19:16:54--  https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving to: 'hmp.parquet'

100%[======================================>] 932,997     --.-K/s   in 0.03s   

2020-02-19 19:16:55 (27.6 MB/s

Dado que este es un aprendizaje supervisado, dividamos nuestros datos en conjunto de entrenamiento (80%) y prueba (20%)

In [2]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

Nuevamente, podemos reutilizar nuestro Pipline de ingeniería de características

In [3]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

Ahora usamos LogisticRegression, un clasificador lineal simple y básico para obtener una línea base de rendimiento de clasificación.

In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer,lr])
model = pipeline.fit(df_train)
prediction = model.transform(df_test)

Si observamos el esquema del marco de datos de predicción, vemos que hay una columna adicional llamada predicción que contiene la mejor conjetura para las predicciones de clase o modelo.

In [5]:
prediction.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- features_norm: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



Vamos a evaluar el rendimiento mediante el uso de una funcionalidad integrada de Apache SparkML

In [6]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
MulticlassClassificationEvaluator().setMetricName("accuracy").evaluate(prediction)

0.2042519260573662

Entonces obtenemos el 20% correcto. Esto no es malo para una línea de base. Tenga en cuenta que adivinar al azar nos daría solo el 7%. Por supuesto que necesitamos mejorar. Es posible que tenga avisos de que estamos tratando con una serie de tiempo aquí. Y no estamos haciendo uso de ese hecho en este momento, ya que miramos cada ejemplo de entrenamiento solo individualmente. Pero esto está bien por ahora. Cursos más avanzados como "Aprendizaje automático avanzado y procesamiento de señales" (https://www.coursera.org/learn/advanced-machine-learning-signal-processing/) le enseñarán cómo mejorar la precisión al 100% utilizando algoritmos como la transformación de Fourier o la transformación wavelet. Pero omita esto por ahora. En la siguiente celda, utilice el clasificador RandomForest (es posible que deba jugar con el parámetro "numTrees") en la celda de código a continuación. Debe obtener una precisión de alrededor del 44%. Puede encontrar más información sobre RandomForest aquí:

https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier